# Using numerical and categorical variables together
In the previous notebooks, we showed the required preprocessing to apply when dealing with numerical and categorical variables. However, we decoupled the process to treat each type individually. In this notebook, we will show how to combine these preprocessing steps.

We will first load the entire adult census dataset.

In [42]:
import pandas as pd
import numpy as np

adult_census = pd.read_csv("/Users/russconte/Adult_Census.csv")
# drop the duplicated column `"education-num"` as stated in the first notebook
adult_census = adult_census.drop(columns="Education-num")

target_name = "Class"
target = adult_census[target_name]

data = adult_census.drop(columns=[target_name])

# Selection based on data types

We will separate categorical and numerical variables using their data types to identify them, as we saw previously that object corresponds to categorical columns (strings). We make use of make_column_selector helper to select the corresponding columns.

In [43]:
from sklearn.compose import make_column_selector as selector

numerical_columns_selector = selector(dtype_exclude=object)
categorical_columns_selector = selector(dtype_include=object)

numerical_columns = numerical_columns_selector(data)
categorical_columns = categorical_columns_selector(data)

# In a more general scenario you should manually introspect the content of your dataframe not to wrongly use make_column_selector.

# Dispatch columns to a specific processor

In the previous sections, we saw that we need to treat data differently depending on their nature (i.e. numerical or categorical).

Scikit-learn provides a ColumnTransformer class which will send specific columns to a specific transformer, making it easy to fit a single predictive model on a dataset that combines both kinds of variables together (heterogeneously typed tabular data).

We first define the columns depending on their data type:

one-hot encoding will be applied to categorical columns. Besides, we use handle_unknown="ignore" to solve the potential issues due to rare categories.
numerical scaling numerical features which will be standardized.
Now, we create our ColumnTransfomer by specifying three values: the preprocessor name, the transformer, and the columns. First, let’s create the preprocessors for the numerical and categorical parts.

In [44]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler

categorical_preprocessor = OneHotEncoder(handle_unknown="ignore")
numerical_preprocessor = StandardScaler()

Now, we create the transformer and associate each of these preprocessors with their respective columns.

In [45]:
from sklearn.compose import ColumnTransformer

preprocessor = ColumnTransformer([
    ('one-hot-encoder', categorical_preprocessor, categorical_columns),
    ('standard_scaler', numerical_preprocessor, numerical_columns)])

A ColumnTransformer does the following:

It splits the columns of the original dataset based on the column names or indices provided. We will obtain as many subsets as the number of transformers passed into the ColumnTransformer.
It transforms each subsets. A specific transformer is applied to each subset: it will internally call fit_transform or transform. The output of this step is a set of transformed datasets.
It then concatenates the transformed datasets into a single dataset.
The important thing is that ColumnTransformer is like any other scikit-learn transformer. In particular it can be combined with a classifier in a Pipeline:

In [46]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline

model = make_pipeline(preprocessor, LogisticRegression(max_iter=500))
model

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('one-hot-encoder',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Workclass', 'Education',
                                                   'Marital-Status',
                                                   'Occupation', 'Relationship',
                                                   'Race', 'Sex',
                                                   'Native-Country']),
                                                 ('standard_scaler',
                                                  StandardScaler(),
                                                  ['Age', 'fnlwgt',
                                                   'Capital-gain',
                                                   'Capital-loss',
                                                   'Hours-per-week'])])),
                ('logisticregression', LogisticRegression(max_iter=500))])

The final model is more complex than the previous models but still follows the same API (the same set of methods that can be called by the user):

the fit method is called to preprocess the data and then train the classifier of the preprocessed data;
the predict method makes predictions on new data;
the score method is used to predict on the test data and compare the predictions to the expected test labels to compute the accuracy.
Let’s start by splitting our data into train and test sets.

In [47]:
from sklearn.model_selection import train_test_split

data_train, data_test, target_train, target_test = train_test_split(
    data, target, random_state=42)

Be aware that we use train_test_split here for didactic purposes, to show the scikit-learn API. In a real setting one might prefer to use cross-validation to also be able to evaluate the uncertainty of our estimation of the generalization performance of a model, as previously demonstrated. Now we can train the model on the train set:

In [48]:
_= model.fit(data_train, target_train)

Now, we can train the model on the train set.

In [49]:
_ = model.fit(data_train, target_train)

Then, we can send the raw dataset straight to the pipeline. Indeed, we do not need to make any manual preprocessing (calling the transform or fit_transform methods) as it will be handled when calling the predict method. As an example, we predict on the five first samples from the test set.

In [50]:
data_test.head()

,Age,Workclass,fnlwgt,Education,Marital-Status,Occupation,Relationship,Race,Sex,Capital-gain,Capital-loss,Hours-per-week,Native-Country
7762,56,Private,33115,HS-grad,Divorced,Other-service,Unmarried,White,Female,0,0,40,United-States
23881,25,Private,112847,HS-grad,Married-civ-spouse,Transport-moving,Own-child,Other,Male,0,0,40,United-States
30507,43,Private,170525,Bachelors,Divorced,Prof-specialty,Not-in-family,White,Female,14344,0,40,United-States
28911,32,Private,186788,HS-grad,Married-civ-spouse,Transport-moving,Husband,White,Male,0,0,40,United-States
19484,39,Private,277886,Bachelors,Married-civ-spouse,Sales,Wife,White,Female,0,0,30,United-States


In [51]:
model.predict(data_test)[:5]

array([' <=50K', ' <=50K', ' >50K', ' <=50K', ' >50K'], dtype=object)

Let's compute the accuracy score on the entire test set:

In [52]:
model.score(data_test, target_test)

0.8583244615510606

# Evaluation of the model with cross-validation

As previously stated, a predictive model should be evaluated by cross-validation. Our model is usable with the cross-validation tools of scikit-learn as any other predictors:

In [53]:
from sklearn.model_selection import cross_validate

cv_results = cross_validate(model, data, target, cv=5)
cv_results

{'fit_time': array([1.14887309, 0.98978686, 1.0173769 , 1.01391792, 1.10054231]),
 'score_time': array([0.02944303, 0.0290072 , 0.02940702, 0.0290482 , 0.02809978]),
 'test_score': array([0.85228785, 0.85085474, 0.84930385, 0.85257985, 0.85667486])}

In [54]:
scores = cv_results["test_score"]
print("The mean cross-validation accuracy is: "
      f"{scores.mean():.3f} ± {scores.std():.3f}")

The mean cross-validation accuracy is: 0.852 ± 0.002


The compound model has a higher predictive accuracy than the two models that used numerical and categorical variables in isolation.

# Fitting a more powerful model

Linear models are nice because they are usually cheap to train, small to deploy, fast to predict and give a good baseline.

However, it is often useful to check whether more complex models such as an ensemble of decision trees can lead to higher predictive performance. In this section we will use such a model called gradient-boosting trees and evaluate its generalization performance. More precisely, the scikit-learn model we will use is called HistGradientBoostingClassifier. Note that boosting models will be covered in more detail in a future module.

For tree-based models, the handling of numerical and categorical variables is simpler than for linear models:

we do <b>not need to scale the numerical features</b>
using an <b>ordinal encoding for the categorical variables</b> is fine even if the encoding results in an arbitrary ordering
Therefore, for HistGradientBoostingClassifier, the preprocessing pipeline is slightly simpler than the one we saw earlier for the LogisticRegression:

In [55]:
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.preprocessing import OrdinalEncoder

categorical_preprocessor = OrdinalEncoder(handle_unknown="use_encoded_value",
                                          unknown_value=-1)

preprocessor = ColumnTransformer([
    ('categorical', categorical_preprocessor, categorical_columns)],
    remainder="passthrough")

model = make_pipeline(preprocessor, HistGradientBoostingClassifier())

Now that we've made our model, we can check the performance of the model:

In [56]:
%%time
_ = model.fit(data_train, target_train)

CPU times: user 16.4 s, sys: 5.79 s, total: 22.2 s
Wall time: 5.99 s


In [57]:
model.score(data_test, target_test)

0.8798624191302924

We can observe that we get significantly higher accuracies with the Gradient Boosting model. This is often what we observe whenever the dataset has a large number of samples and limited number of informative features (e.g. less than 1000) with a mix of numerical and categorical variables.

This explains why Gradient Boosted Machines are very popular among datascience practitioners who work with tabular data.

# Repeat the gradient boosting process with the diamonds data set:

In [58]:
diamonds = pd.read_csv('/Users/russconte/diamonds.csv')

In [59]:
target = diamonds["cut"]
data = diamonds.drop(columns='cut')
diamonds.head()

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


In [60]:
from sklearn.compose import make_column_selector as selector

numerical_columns_selector = selector(dtype_exclude=object)
categorical_columns_selector = selector(dtype_include=object)

numerical_columns = numerical_columns_selector(data)
categorical_columns = categorical_columns_selector(data)

In [61]:
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.preprocessing import OrdinalEncoder

categorical_preprocessor = OrdinalEncoder(handle_unknown="use_encoded_value",
                                          unknown_value=-1)

preprocessor = ColumnTransformer([
    ('categorical', categorical_preprocessor, categorical_columns)],
    remainder="passthrough")

model = make_pipeline(preprocessor, HistGradientBoostingClassifier())

In [62]:
from sklearn.model_selection import cross_validate

cv_results = cross_validate(model, data, target, cv=5)
cv_results

{'fit_time': array([35.51382399, 41.32982802, 41.22256899, 33.746171  , 37.90856791]),
 'score_time': array([0.41495395, 0.48189807, 0.27926397, 0.27796197, 0.29607034]),
 'test_score': array([0.59139785, 0.38107156, 0.24416018, 0.56349648, 0.67927327])}

In [63]:
scores = cv_results["test_score"]
print("The mean cross-validation accuracy is: "
      f"{scores.mean():.3f} ± {scores.std():.3f}")

The mean cross-validation accuracy is: 0.492 ± 0.157


This is 0.005 better than the previous version using logistic regression which scored at 0.481 ± 0.013

In this notebook we:

used a ColumnTransformer to apply different preprocessing for categorical and numerical variables;
used a pipeline to chain the ColumnTransformer preprocessing and logistic regression fitting;
saw that gradient boosting methods can outperform linear models.